In [71]:
import sqlglot.expressions as se

In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import sqlglot

In [69]:
sql = 'SELECT a, avg(b), sdfg(b), agsh(a, b) FROM mytable WHERE a > 10 AND a < 15 OR a > 1000 GROUP BY a ORDER BY a, avg(b)'

In [70]:
parsed = sqlglot.parse(sql, 'postgres')[0]  # sqlglot.parse(sql) returns list, every element is sql statement if I understand correctly
parsed

(SELECT expressions: 
  (COLUMN this: 
    (IDENTIFIER this: a, quoted: False)), 
  (AVG this: 
    (COLUMN this: 
      (IDENTIFIER this: b, quoted: False))), 
  (ANONYMOUS this: sdfg, expressions: 
    (COLUMN this: 
      (IDENTIFIER this: b, quoted: False))), 
  (ANONYMOUS this: agsh, expressions: 
    (COLUMN this: 
      (IDENTIFIER this: a, quoted: False)), 
    (COLUMN this: 
      (IDENTIFIER this: b, quoted: False))), from: 
  (FROM expressions: 
    (TABLE this: 
      (IDENTIFIER this: mytable, quoted: False))), where: 
  (WHERE this: 
    (OR this: 
      (AND this: 
        (GT this: 
          (COLUMN this: 
            (IDENTIFIER this: a, quoted: False)), expression: 
          (LITERAL this: 10, is_string: False)), expression: 
        (LT this: 
          (COLUMN this: 
            (IDENTIFIER this: a, quoted: False)), expression: 
          (LITERAL this: 15, is_string: False))), expression: 
      (GT this: 
        (COLUMN this: 
          (IDENTIFIER this: a, quo

In [78]:
parsed.find(se.Group).expressions

[(COLUMN this: 
   (IDENTIFIER this: a, quoted: False))]

In [75]:
list(parsed.find_all(se.Anonymous))[1].expressions

[(COLUMN this: 
   (IDENTIFIER this: a, quoted: False)),
 (COLUMN this: 
   (IDENTIFIER this: b, quoted: False))]

In [66]:
parsed.expressions

[(COLUMN this: 
   (IDENTIFIER this: a, quoted: False)),
 (AVG this: 
   (COLUMN this: 
     (IDENTIFIER this: b, quoted: False)))]

In [33]:
type(parsed)

sqlglot.expressions.Select

In [57]:
parsed.args['order'].find(sqlglot.expressions.Ordered).args['nulls_first']

False

In [34]:
parsed.args['from']

(FROM expressions: 
  (TABLE this: 
    (IDENTIFIER this: mytable, quoted: False)))

In [35]:
parsed.args['from'].expressions

[(TABLE this: 
   (IDENTIFIER this: mytable, quoted: False))]

In [36]:
parsed.args['from'].find(sqlglot.expressions.Identifier).this

'mytable'

In [37]:
for x in parsed.expressions:
    print(type(x))

<class 'sqlglot.expressions.Column'>
<class 'sqlglot.expressions.Avg'>


In [38]:
from sqlglot.expressions import Where

where = parsed.find(Where)
where

(WHERE this: 
  (OR this: 
    (AND this: 
      (GT this: 
        (COLUMN this: 
          (IDENTIFIER this: a, quoted: False)), expression: 
        (LITERAL this: 10, is_string: False)), expression: 
      (LT this: 
        (COLUMN this: 
          (IDENTIFIER this: a, quoted: False)), expression: 
        (LITERAL this: 15, is_string: False))), expression: 
    (GT this: 
      (COLUMN this: 
        (IDENTIFIER this: a, quoted: False)), expression: 
      (LITERAL this: 1000, is_string: False))))

In [39]:
where

(WHERE this: 
  (OR this: 
    (AND this: 
      (GT this: 
        (COLUMN this: 
          (IDENTIFIER this: a, quoted: False)), expression: 
        (LITERAL this: 10, is_string: False)), expression: 
      (LT this: 
        (COLUMN this: 
          (IDENTIFIER this: a, quoted: False)), expression: 
        (LITERAL this: 15, is_string: False))), expression: 
    (GT this: 
      (COLUMN this: 
        (IDENTIFIER this: a, quoted: False)), expression: 
      (LITERAL this: 1000, is_string: False))))

In [40]:
and_ = where.this
and_

(OR this: 
  (AND this: 
    (GT this: 
      (COLUMN this: 
        (IDENTIFIER this: a, quoted: False)), expression: 
      (LITERAL this: 10, is_string: False)), expression: 
    (LT this: 
      (COLUMN this: 
        (IDENTIFIER this: a, quoted: False)), expression: 
      (LITERAL this: 15, is_string: False))), expression: 
  (GT this: 
    (COLUMN this: 
      (IDENTIFIER this: a, quoted: False)), expression: 
    (LITERAL this: 1000, is_string: False)))

In [41]:
and_

(OR this: 
  (AND this: 
    (GT this: 
      (COLUMN this: 
        (IDENTIFIER this: a, quoted: False)), expression: 
      (LITERAL this: 10, is_string: False)), expression: 
    (LT this: 
      (COLUMN this: 
        (IDENTIFIER this: a, quoted: False)), expression: 
      (LITERAL this: 15, is_string: False))), expression: 
  (GT this: 
    (COLUMN this: 
      (IDENTIFIER this: a, quoted: False)), expression: 
    (LITERAL this: 1000, is_string: False)))

In [42]:
for k, v in parsed.args.items():
    print(k, v)

hint None
distinct None
expressions [(COLUMN this: 
  (IDENTIFIER this: a, quoted: False)), (AVG this: 
  (COLUMN this: 
    (IDENTIFIER this: b, quoted: False)))]
limit None
from FROM mytable
where WHERE a > 10 AND a < 15 OR a > 1000
group GROUP BY a
order ORDER BY a NULLS LAST


In [43]:
from sqlglot import Expression
from sqlglot.helper import ensure_collection

for k, v in parsed.args.items():
    for node in ensure_collection(v):
        if isinstance(node, Expression):
            print(k, node)

expressions a
expressions AVG(b)
from FROM mytable
where WHERE a > 10 AND a < 15 OR a > 1000
group GROUP BY a
order ORDER BY a NULLS LAST


In [44]:
from sqlglot import Expression
from sqlglot.helper import ensure_collection

start_node = where.this
print(type(start_node))
for k, v in start_node.args.items():
    for node in ensure_collection(v):
        if isinstance(node, Expression):
            print(k, node)

<class 'sqlglot.expressions.Or'>
this a > 10 AND a < 15
expression a > 1000


In [45]:
#node: Expression
for node in where.this.dfs():
    print(node[0])


a > 10 AND a < 15 OR a > 1000
a > 10 AND a < 15
a > 10
a
a
10
a < 15
a
a
15
a > 1000
a
a
1000


In [46]:
where

(WHERE this: 
  (OR this: 
    (AND this: 
      (GT this: 
        (COLUMN this: 
          (IDENTIFIER this: a, quoted: False)), expression: 
        (LITERAL this: 10, is_string: False)), expression: 
      (LT this: 
        (COLUMN this: 
          (IDENTIFIER this: a, quoted: False)), expression: 
        (LITERAL this: 15, is_string: False))), expression: 
    (GT this: 
      (COLUMN this: 
        (IDENTIFIER this: a, quoted: False)), expression: 
      (LITERAL this: 1000, is_string: False))))

In [47]:
import where_parser

where_func = where_parser.parse_where_simple(where.this)
where_func


def _parse_where_simple_inner(a):
    return a > 10 and a < 15 or a > 1000



<function where_parser._parse_where_simple_inner(a)>

In [48]:
where_func(1210)

True

In [58]:
sql2 = 'SELECT 1'

In [59]:
parsed2 = sqlglot.parse(sql2, 'postgres')[0]
parsed2

(SELECT expressions: 
  (LITERAL this: 1, is_string: False))

In [65]:
print(parsed2.find(se.Where))

None
